## Importing libraries

In [1]:
import pandas as pd
import numpy as np 
from scipy.stats import mstats
from sklearn.preprocessing import LabelEncoder
from autoviml.Auto_ViML import Auto_ViML


Imported Auto_ViML version = 0.1.463. 
             m, feats, trainm, testm = Auto_ViML(train, target, test, 
                                    sample_submission='',
                                    scoring_parameter='',
                                    hyper_param='GS',feature_reduction=True,
                                     Boosting_Flag=None,Binning_Flag=False,
                                    Add_Poly=0, Stacking_Flag=False,Imbalanced_Flag=False, 
                                    verbose=0)
            
To remove previous versions, perform 'pip uninstall autoviml'


In [22]:
url1 = '../dataset/dataset_treino.csv'
url2 = '../dataset/dataset_teste.csv'
url3 = '../dataset/sample_submission.csv'
train = pd.read_csv(url1,index_col=None)
test = pd.read_csv(url2,index_col=None)
sample_submission = pd.read_csv(url3,index_col=None)
print(train.shape,test.shape,sample_submission.shape)

(114321, 133) (114393, 132) (114393, 2)


In [23]:
# Juntando os dois dataset (treino e teste)
df = train.append(test)
df.shape

(228714, 133)

# Feature Engineering

In [26]:
# Tratando os dados outliers com Winsorize
for col in df.columns:
    if df[col].dtype !='object':
        if col in ['ID','target']:
            continue
        df[col] = mstats.winsorize(df[col], limits=[0.05, 0.05])[0]
        df[col].fillna(df[col].mean(),inplace=True)
    else:
        df[col].fillna('X',inplace=True)
        
# Função para criação de novas features, agrupando por algumas colunas
def ft(data):
    df = pd.DataFrame()

    numerical_cols = data.dtypes[data.dtypes != "object"].index

    for col in numerical_cols:
        if col in ['ID','target','v2']:
            continue
        
        df[col + '_mean']   = data.groupby(['v2'])[col].mean()
        df[col + '_median'] = data.groupby(['v2'])[col].median()
        df[col + '_max']    = data.groupby(['v2'])[col].max()
        df[col + '_min']    = data.groupby(['v2'])[col].min()
        df[col + '_std'] = data.groupby(['v2'])[col].std()

    return df
    
new_df = ft(df)
new_df = new_df.reset_index()
new_df = pd.merge(df, new_df, on='v2',how='left')

In [28]:
le = LabelEncoder()

for col in categorical_feats:
    le.fit(np.unique(list(new_df[col].values)))
    new_df[col] = le.transform(new_df[col])

            
#new_df['v125'] = new_df['v125'].astype('category')
#new_df['v22']  = new_df['v22'].astype('category')
#new_df['v56']  = new_df['v56'].astype('category')

#new_df["v125"] = new_df["v125"].cat.codes
#new_df["v22"]  = new_df["v22"].cat.codes
#new_df["v56"]  = new_df["v56"].cat.codes

In [32]:
numerical_feats = new_df.dtypes[new_df.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = new_df.dtypes[new_df.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  688
Number of Categorical features:  0


In [29]:
new_df.shape

(228714, 688)

In [12]:
columns_dummies = ['v3','v107','v110','v112','v113','v24','v30','v31','v47','v52','v66','v71','v74','v75','v79','v91']

new_df = pd.get_dummies(data=new_df, columns=columns_dummies)

# Convertendo as colunas uint8 to int
for col in new_df.columns:
    if new_df[col].dtype =='uint8':
        new_df[col] = new_df[col].astype('int')
        
new_df.shape

(228714, 717)

# Modelo Machine Learning

In [33]:
sample_submission = ''
scoring_parameter = 'logloss'
target='target'

In [34]:
train_df = new_df[new_df['target'].notnull()]
test_df = new_df[new_df['target'].isnull()]

train_df.shape, test_df.shape

((114321, 688), (114393, 688))

In [35]:
#### If Boosting_Flag = True => XGBoost, False=>ExtraTrees, None=>Linear Model
m, feats, trainm, testm = Auto_ViML(train_df, target, test_df, sample_submission,
                                    scoring_parameter=scoring_parameter,
                                    hyper_param='GS',
                                    feature_reduction=True,
                                    Boosting_Flag=False,
                                    Binning_Flag=False,
                                    Add_Poly=0, 
                                    Stacking_Flag=False,                                    
                                    Imbalanced_Flag=True, 
                                    verbose=1)

Train (Size: 114321,688) has Single_Label with target: ['target']
"
 ################### Binary-Class ##################### " 
Shuffling the data set before training
 Class  -> Counts -> Percent
   1.0:   87021  ->   76.1%
   0.0:   27300  ->   23.9%
Selecting 2-Class Classifier...
    Using GridSearchCV for Hyper Parameter tuning...
    Target target is already numeric. No transformation done.
Classifying variables in data set...
    Number of Numeric Columns =  0
    Number of Integer-Categorical Columns =  19
    Number of String-Categorical Columns =  0
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  0
    Number of Discrete String Columns =  0
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  1
    Number of Columns to Delete =  667
    687 Predictors classified...
        This does not include the Target column(s)
    668 variables removed since th

UnboundLocalError: local variable 'best_score' referenced before assignment